In [1]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk

In [2]:
class HotelManagementApp(tk.Toplevel):
    def __init__(self, master):
        self.master = master
        self.master.title("Hotel Management System")
        self.master.geometry("600x400")
        
        # Load and display background image
        bg_image = Image.open('bg.jpg')
        bg_photo = ImageTk.PhotoImage(bg_image)
        bg_label = tk.Label(self, image=bg_photo)
        bg_label.image = bg_photo
        bg_label.place(x=0, y=0, relwidth=1, relheight=1)


        
        # Create a canvas
        self.canvas = tk.Canvas(self.master)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        # Add a scrollbar to the canvas
        self.scrollbar = tk.Scrollbar(self.master, orient=tk.VERTICAL, command=self.canvas.yview)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Configure the canvas to use the scrollbar
        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        self.canvas.bind('<Configure>', lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all")))

        # Create a frame inside the canvas
        self.frame = tk.Frame(self.canvas)
        self.canvas.create_window((0, 0), window=self.frame, anchor="nw")

        self.rooms = {}
        for i in range(101, 151):
            room_type = "AC" if i % 2 != 0 else "Non-AC"
            self.rooms[str(i)] = {"status": "available", "type": room_type, "guest_info": []}




        self.room_label = tk.Label(self.frame, text="Room Number:")
        self.room_label.grid(row=0, column=0, padx=10, pady=10)

        self.room_entry = tk.Entry(self.frame)
        self.room_entry.grid(row=0, column=1, padx=10, pady=10)

        self.check_button = tk.Button(self.frame, text="Check Availability", command=self.check_availability)
        self.check_button.grid(row=0, column=2, padx=10, pady=10)

        self.result_label = tk.Label(self.frame, text="")
        self.result_label.grid(row=1, column=0, columnspan=3, padx=10, pady=10)

        self.accept_button = tk.Button(self.frame, text="Accept Guest", command=self.accept_guest)
        self.accept_button.grid(row=2, column=0, padx=10, pady=10)

        self.decline_button = tk.Button(self.frame, text="Decline Guest", command=self.decline_guest)
        self.decline_button.grid(row=2, column=1, padx=10, pady=10)

        self.ac_rooms_label = tk.Label(self.frame, text="")
        self.ac_rooms_label.grid(row=3, column=0, columnspan=3, padx=10, pady=10)

        self.non_ac_rooms_label = tk.Label(self.frame, text="")
        self.non_ac_rooms_label.grid(row=4, column=0, columnspan=3, padx=10, pady=10)

        self.total_guest_label = tk.Label(self.frame, text="")
        self.total_guest_label.grid(row=5, column=0, columnspan=3, padx=10, pady=10)

        self.available_rooms_label = tk.Label(self.frame, text="Available Rooms:")
        self.available_rooms_label.grid(row=6, column=0, padx=10, pady=10)

        self.available_rooms_listbox = tk.Listbox(self.frame, width=30, height=10)
        self.available_rooms_listbox.grid(row=7, column=0, columnspan=3, padx=10, pady=5)

        self.update_room_status_listbox()

        self.display_button = tk.Button(self.frame, text="Display Alloted Rooms", command=self.display_alloted_rooms)
        self.display_button.grid(row=8, column=0, columnspan=3, padx=10, pady=10)

    def check_availability(self):
        room_number = self.room_entry.get()
        if room_number in self.rooms:
            status = self.rooms[room_number]["status"]
            room_type = self.rooms[room_number]["type"]
            self.result_label.config(text=f"Room {room_number} is {status}. Type: {room_type}")
        else:
            self.result_label.config(text=f"Room {room_number} does not exist.")

    def accept_guest(self):
        room_number = self.room_entry.get()
        if room_number in self.rooms and self.rooms[room_number]["status"] == "available":
            self.get_guest_information(room_number)
        else:
            messagebox.showwarning("Invalid Room", f"Room {room_number} is not available.")

    def accept_guest_internal(self, room_number, guest_info):
        if room_number in self.rooms and self.rooms[room_number]["status"] == "available":
            self.rooms[room_number]["status"] = "occupied"
            self.rooms[room_number]["guest_info"].append(guest_info)
            self.update_total_guests()
            self.update_available_rooms()
            self.update_room_status_listbox()

    def decline_guest(self):
        room_number = self.room_entry.get()
        if room_number in self.rooms and self.rooms[room_number]["status"] == "occupied":
            self.rooms[room_number]["status"] = "available"
            self.update_total_guests()
            self.update_available_rooms()
            self.update_room_status_listbox()
        else:
            messagebox.showwarning("Invalid Room", f"No guest in Room {room_number}.")

    def update_total_guests(self):
        total_guests = sum(len(room["guest_info"]) for room in self.rooms.values() if room["status"] == "occupied")
        self.total_guest_label.config(text=f"Total Guests: {total_guests}")

    def update_available_rooms(self):
        ac_rooms_count = sum(1 for room in self.rooms.values() if room["type"] == "AC" and room["status"] == "available")
        non_ac_rooms_count = sum(1 for room in self.rooms.values() if room["type"] == "Non-AC" and room["status"] == "available")
        self.ac_rooms_label.config(text=f"Available AC Rooms: {ac_rooms_count}")
        self.non_ac_rooms_label.config(text=f"Available Non-AC Rooms: {non_ac_rooms_count}")

    def update_room_status_listbox(self):
        self.available_rooms_listbox.delete(0, tk.END)
        for room_number, room_info in self.rooms.items():
            if room_info["status"] == "available":
                room_type = room_info["type"]
                self.available_rooms_listbox.insert(tk.END, f"Room {room_number} ({room_type})")

    def get_guest_information(self, room_number):
        GuestInfoWindow(self.master, room_number, self)

    def ask_payment_option(self, room_number):
        PaymentWindow(self.master, room_number, self)

    def complete_payment(self, room_number, payment_method):
        if payment_method == "Cash":
            messagebox.showinfo("Payment Accepted", "Payment completed in cash. Thank you!")
        else:
            messagebox.showinfo("Payment Accepted", "Payment completed online. Thank you!")

    def display_alloted_rooms(self):
        self.alloted_rooms_listbox = tk.Listbox(self.frame, width=40, height=10)
        self.alloted_rooms_listbox.grid(row=9, column=0, columnspan=3, padx=10, pady=10)

        def show_guest_information(room_number):
            guest_info_list = self.rooms[room_number]["guest_info"]
            info_str = ""
            for guest_info in guest_info_list:
                info_str += f"Name: {guest_info['name']}\nAddress: {guest_info['address']}\nAadhar Number: {guest_info['aadhar_number']}\nPhone Number: {guest_info['phone_number']}\nExtra Number: {guest_info['extra_number']}\n\n"
            if info_str:
                messagebox.showinfo("Guest Information", info_str)
            else:
                messagebox.showinfo("Guest Information", "No guest information available for this room.")

        def create_button(room_number):
            return tk.Button(self.frame, text=f"Room {room_number}", command=lambda: show_guest_information(room_number))

        alloted_rooms = [room_number for room_number, room_info in self.rooms.items() if room_info["status"] == "occupied"]
        if alloted_rooms:
            for idx, room_number in enumerate(alloted_rooms):
                self.alloted_rooms_listbox.insert(tk.END, f"Room {room_number}")
                room_button = create_button(room_number)
                room_button.grid(row=10 + idx, column=0, columnspan=3, padx=10, pady=10)
        else:
            messagebox.showinfo("Alloted Rooms", "No rooms are currently alloted.")
